In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import optuna
from tensorflow.keras.regularizers import l2
import optuna
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout, Attention, Concatenate, Reshape
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.regularizers import l2
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import joblib
from prophet import Prophet
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import optuna
from tensorflow.keras.regularizers import l2
import optuna
import joblib
import xgboost as xgb
import shap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor

file_path = '/content/lstm_exch.xlsx'
df = pd.read_excel(file_path, sheet_name='Exch')
df['Date'] = pd.to_datetime(df['Date'])

# --------------------------
# Feature Engineering (no leakage)
# --------------------------
import numpy as np
import pandas as pd
from prophet import Prophet

def feature_engineering(df):
    df = df.copy()

    # Ensure datetime format
    df['Date'] = pd.to_datetime(df['Date'])
    df.sort_values('Date', inplace=True)

    # Lag Features
    df['USD/PKR_lag1'] = df['USD/PKR'].shift(1)
    df['USD/PKR_lag3'] = df['USD/PKR'].shift(3)
    df['USD/PKR_lag7'] = df['USD/PKR'].shift(7)
    df['USD_EUR_lag1'] = df['USD/EUR'].shift(1)
    df['USD_EUR_lag2'] = df['USD/EUR'].shift(3)
    df['USD_JPY_lag3'] = df['USD/JPY'].shift(1)


    # Rolling statistics
    df['roll_mean_3'] = df['USD/PKR'].rolling(window=3).mean()
    df['roll_std_3'] = df['USD/PKR'].rolling(window=3).std()

    # Momentum
    df['pkr_diff'] = df['USD/PKR'].diff()
    df['pkr_pct'] = df['USD/PKR'].pct_change()

    # Other currency relationships
    df['usd_jpy_diff'] = df['USD/JPY'].diff()
    df['usd_eur_diff'] = df['USD/EUR'].diff()

    # Time-based
    df['days_since_start'] = (df['Date'] - df['Date'].min()).dt.days

    df['month'] = df['Date'].dt.month
    df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
    df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)

    # Prophet forecast
    prophet_df = df[['Date', 'USD/PKR']].rename(columns={'Date': 'ds', 'USD/PKR': 'y'})
    prophet_model = Prophet(yearly_seasonality=True, daily_seasonality=False)
    prophet_model.fit(prophet_df)
    forecast = prophet_model.predict(prophet_df)
    df['prophet_pred'] = forecast['yhat'].shift(1)
    df['prophet_residual'] = df['USD/PKR'] - df['prophet_pred'].shift(1)

    # Momentum
    df['pkr_momentum_7'] = df['USD/PKR'] - df['USD/PKR'].shift(7)

# Then include 'prophet_pred_lagged' in your features before scaling and building sequences
   # Drop rows with any NaN values
    df.dropna(inplace=True)

    return df
"""
df = feature_engineering(df)

# ------------------ 1. Prepare the data ------------------
y = df['USD/PKR']  # Target column
X = df.drop(['USD/PKR','Date','month'], axis=1)  # Feature columns

# Step 1: Remove Multicollinearity
def remove_multicollinearity(df, threshold=0.95):
    corr = df.corr().abs()
    upper = corr.where(np.triu(np.ones(corr.shape), k=1).astype(bool))
    to_drop = [column for column in upper.columns if any(upper[column] > threshold)]
    print(f"Removed {len(to_drop)} highly correlated features: {to_drop}")
    return df.drop(columns=to_drop), to_drop

# Step 2: Compare SHAP Importances
def compare_shap_importances(X, y, top_n=10):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # XGBoost model + SHAP
    xgb_model = xgb.XGBRegressor(n_estimators=100, max_depth=6, learning_rate=0.1, random_state=42)
    xgb_model.fit(X_train, y_train)
    explainer_xgb = shap.Explainer(xgb_model)  # Uses TreeExplainer by default
    shap_values_xgb = explainer_xgb(X_train)
    xgb_shap = pd.DataFrame({
        'feature': X.columns,
        'xgb_mean_abs_shap': np.abs(shap_values_xgb.values).mean(axis=0)
    })

    # Neural Network (MLP)
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    nn_model = MLPRegressor(hidden_layer_sizes=(64, 64), max_iter=500, random_state=42)
    nn_model.fit(X_train_scaled, y_train)

    # SHAP for NN (KernelExplainer - expensive, so sample)
    sample_X = shap.sample(X_train_scaled, 100, random_state=42)
    explainer_nn = shap.KernelExplainer(nn_model.predict, sample_X)
    shap_values_nn = explainer_nn.shap_values(X_train_scaled[:50], nsamples=100)

    nn_shap = pd.DataFrame({
        'feature': X.columns,
        'nn_mean_abs_shap': np.abs(shap_values_nn).mean(axis=0)
    })

    # Merge importances
    comparison_df = pd.merge(xgb_shap, nn_shap, on='feature')
    comparison_df = comparison_df.sort_values('xgb_mean_abs_shap', ascending=False)

    # Plot
    comparison_df.set_index('feature')[['xgb_mean_abs_shap', 'nn_mean_abs_shap']].head(top_n).plot(
        kind='barh', figsize=(10, 6), title='XGBoost vs NN SHAP Importance'
    )
    plt.gca().invert_yaxis()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    return comparison_df

# Step 3: Select Top Features
def select_top_features(shap_df, top_k=10, method='average'):
    if method == 'average':
        shap_df['avg_shap'] = shap_df[['xgb_mean_abs_shap', 'nn_mean_abs_shap']].mean(axis=1)
        top_feats = shap_df.sort_values('avg_shap', ascending=False)['feature'].head(top_k).tolist()
    elif method == 'intersection':
        top_xgb = shap_df.sort_values('xgb_mean_abs_shap', ascending=False)['feature'].head(top_k)
        top_nn = shap_df.sort_values('nn_mean_abs_shap', ascending=False)['feature'].head(top_k)
        top_feats = list(set(top_xgb).intersection(set(top_nn)))
    else:
        raise ValueError("Method must be 'average' or 'intersection'")

    print(f"\nSelected top {len(top_feats)} features ({method}): {top_feats}")
    return top_feats

# Final Pipeline
def shap_feature_selection_pipeline(X, y, multicollinearity_threshold=0.95, top_k=10, method='average'):
    print("\n--- Running SHAP-Based Feature Selection Pipeline ---")

    # Step 1: Multicollinearity
    X_reduced, dropped_cols = remove_multicollinearity(X, threshold=multicollinearity_threshold)

    # Step 2: SHAP comparison
    shap_df = compare_shap_importances(X_reduced, y, top_n=top_k)

    # Step 3: Feature selection
    selected_features = select_top_features(shap_df, top_k=top_k, method=method)

    # Return selected dataset
    return X_reduced[selected_features], selected_features, shap_df, dropped_cols
# RUN
X_selected, selected_feats, shap_df, dropped = shap_feature_selection_pipeline(
    X, y,
    multicollinearity_threshold=0.95,
    top_k=10,
    method='average'
)

# -- MODELING --

df.sort_values('Date', inplace=True)
df.drop('Date',axis=1)

# Selecting features
data = df[[
    'USD/PKR',
    'USD/PKR_lag1',
    'USD/JPY',
    'USD/EUR',
    'days_since_start'
]].astype(float)

scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)

# Create sequences
def create_sequences(data, lookback):
    X, y = [], []
    for i in range(lookback, len(data)):
        X.append(data[i - lookback:i])
        y.append(data[i, 0])  # Target is USD/PKR
    return np.array(X), np.array(y)

# 1.37 best_params = {'batch_size': 16, 'num_layers': 1, 'lookback': 70, 'units': 72, 'dropout': 0.015182762252143527, 'lr': 0.004446771977416189, 'l2_reg_lstm': 1.2715967548965089e-06, 'early_patience': 10, 'early_min_delta': 6.23793908071683e-05, 'reduce_factor': 0.8880892095511763, 'reduce_patience': 3, 'min_lr': 5.595165634673799e-06}
# 1.39 {'batch_size': 16, 'num_layers': 1, 'lookback': 95, 'units': 72, 'dropout': 0.020354140678498382, 'lr': 0.007055226563672889, 'l2_reg_lstm': 7.540667771887033e-07, 'early_patience': 10, 'early_min_delta': 1.1366211591681307e-05, 'reduce_factor': 0.7841208938505344, 'reduce_patience': 2, 'min_lr': 3.774349882958255e-05}
# value 1.41 best_params = {'batch_size': 16, 'num_layers': 1, 'lookback': 70, 'units': 72, 'dropout': 0.015182762252143527, 'lr': 0.004446771977416189, 'l2_reg_lstm': 1.2715967548965089e-06, 'early_patience': 10, 'early_min_delta': 6.23793908071683e-05, 'reduce_factor': 0.8880892095511763, 'reduce_patience': 3, 'min_lr': 5.595165634673799e-06}
# value: 1.40  {'batch_size': 16, 'num_layers': 1, 'lookback': 95, 'units': 72, 'dropout': 0.020354140678498382, 'lr': 0.007055226563672889, 'l2_reg_lstm': 7.540667771887033e-07, 'early_patience': 10, 'early_min_delta': 1.1366211591681307e-05, 'reduce_factor': 0.7841208938505344, 'reduce_patience': 2, 'min_lr': 3.774349882958255e-05}

# value: 1.3768831331105493 and parameters: {'batch_size': 16, 'num_layers': 1, 'lookback': 108, 'units': 57, 'dropout': 0.011810305033639949, 'lr': 0.006185124160360622, 'l2_reg_lstm': 2.0623626782630574e-06, 'early_patience': 13, 'early_min_delta': 5.408021753464203e-05, 'reduce_factor': 0.6632453018415295, 'reduce_patience': 3, 'min_lr': 4.200249724501052e-06}
def objective(trial):
    # Hyperparameters
    batch_size = trial.suggest_categorical('batch_size', [16])
    num_layers = trial.suggest_int('num_layers', 1,1)
    lookback = trial.suggest_int('lookback', 65,120 )
    units = trial.suggest_int('units', 55,75)
    dropout = trial.suggest_float('dropout',  0.01, 0.1)
    lr = trial.suggest_float('lr', 0.004, 0.09, log=True)
    l2_reg_lstm = trial.suggest_float('l2_reg_lstm', 1e-7, 1e-6, log=True)
    #l2_reg_dense = trial.suggest_float('l2_reg_dense', 1e-8, 1e-4, log=True)


    # Callbacks tuning
    early_patience = trial.suggest_int("early_patience", 5, 10)
    early_min_delta = trial.suggest_float("early_min_delta", 1e-5, 1e-2, log=True)
    reduce_factor = trial.suggest_float("reduce_factor", 0.1, 0.9)
    reduce_patience = trial.suggest_int("reduce_patience", 1,3 )
    min_lr = trial.suggest_float("min_lr", 1e-6, 1e-4, log=True)

    # Prepare data
    X, y = create_sequences(scaled_data, lookback)
    train_size = int(len(X) * 0.8)
    X_train, X_test = X[:train_size], X[train_size:]
    y_train, y_test = y[:train_size], y[train_size:]

    # Build model
    inputs = Input(shape=(X_train.shape[1], X_train.shape[2]))
    x = inputs
    for i in range(num_layers):
        return_seq =  i < num_layers - 1
        x = LSTM(units=units, return_sequences=return_seq,
                 kernel_regularizer=l2(l2_reg_lstm))(x)
        if dropout > 0:
            x = Dropout(dropout)(x)
    output = Dense(1)(x)
    model = Model(inputs, output)
    model.compile(optimizer=Adam(learning_rate=lr), loss='mae')

    callbacks = [
        EarlyStopping(monitor='val_loss', patience=early_patience,
                      restore_best_weights=True, min_delta=early_min_delta),
        ReduceLROnPlateau(monitor='val_loss', factor=reduce_factor,
                          patience=reduce_patience, min_lr=min_lr)
    ]

    model.fit(X_train, y_train, validation_data=(X_test, y_test),
              epochs=50, batch_size=batch_size, verbose=1, callbacks=callbacks)

    y_pred = model.predict(X_test)
    y_pred_inv = scaler.inverse_transform(np.concatenate((y_pred, X_test[:, -1, 1:]), axis=1))[:, 0]
    y_test_inv = scaler.inverse_transform(np.concatenate((y_test.reshape(-1, 1), X_test[:, -1, 1:]), axis=1))[:, 0]
    rmse = np.sqrt(mean_squared_error(y_test_inv, y_pred_inv))
    return rmse

# Run optimization
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

print("Best Hyperparameters:", study.best_params)
print("Best RMSE:", study.best_value)

# Extract best parameters
best_params = study.best_params

# Recreate sequences
lookback = best_params['lookback']
X, y = create_sequences(scaled_data, lookback)
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Build model with or without attention
inputs = Input(shape=(X_train.shape[1], X_train.shape[2]))
x = inputs
for i in range(best_params['num_layers']):
    return_seq =  i < best_params['num_layers'] - 1
    x = LSTM(units=best_params['units'], return_sequences=return_seq,
             kernel_regularizer=l2(best_params['l2_reg_lstm']))(x)
    if best_params['dropout'] > 0:
        x = Dropout(best_params['dropout'])(x)
output = Dense(1)(x)
final_model = Model(inputs, output)
final_model.compile(optimizer=Adam(learning_rate=best_params['lr']), loss='mae')

# Callbacks
callbacks = [
    EarlyStopping(monitor='val_loss', patience=best_params['early_patience'],
                  restore_best_weights=True, min_delta=best_params['early_min_delta']),
    ReduceLROnPlateau(monitor='val_loss', factor=best_params['reduce_factor'],
                      patience=best_params['reduce_patience'], min_lr=best_params['min_lr'])
]

# Train
history = final_model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=100,
    batch_size=best_params['batch_size'],
    callbacks=callbacks,
    verbose=1
)
from sklearn.metrics import mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt
import joblib
import numpy as np

# === PREDICTION on TEST ===
y_pred_test = final_model.predict(X_test)
y_pred_test_inv = scaler.inverse_transform(np.concatenate((y_pred_test, X_test[:, -1, 1:]), axis=1))[:, 0]
y_test_inv = scaler.inverse_transform(np.concatenate((y_test.reshape(-1, 1), X_test[:, -1, 1:]), axis=1))[:, 0]

# === PREDICTION on TRAIN ===
y_pred_train = final_model.predict(X_train)
y_pred_train_inv = scaler.inverse_transform(np.concatenate((y_pred_train, X_train[:, -1, 1:]), axis=1))[:, 0]
y_train_inv = scaler.inverse_transform(np.concatenate((y_train.reshape(-1, 1), X_train[:, -1, 1:]), axis=1))[:, 0]

# === METRICS ===
rmse_test = np.sqrt(mean_squared_error(y_test_inv, y_pred_test_inv))
mae_test = mean_absolute_error(y_test_inv, y_pred_test_inv)

rmse_train = np.sqrt(mean_squared_error(y_train_inv, y_pred_train_inv))
mae_train = mean_absolute_error(y_train_inv, y_pred_train_inv)

print(f"Test RMSE: {rmse_test:.4f}")
print(f"Test MAE : {mae_test:.4f}")
print(f"Train RMSE: {rmse_train:.4f}")
print(f"Train MAE : {mae_train:.4f}")

# === SAVE FINAL MODEL ===
final_model.save("final_lstm_model.h5")
joblib.dump(scaler, "scaler.save")

# === LOSS PLOT ===
plt.figure(figsize=(8, 5))
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title("Loss Curve")
plt.xlabel("Epoch")
plt.ylabel("MAE Loss")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig("loss_curve.png")
plt.show()

# === PLOT PREDICTIONS: TEST ===
plt.figure(figsize=(12, 6))
plt.plot(y_test_inv, label='Actual')
plt.plot(y_pred_test_inv, label='Predicted')
plt.title("Test Set: Predicted vs Actual USD/PKR")
plt.xlabel("Time")
plt.ylabel("USD/PKR Rate")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig("prediction_vs_actual_test.png")
plt.show()

# === PLOT PREDICTIONS: TRAIN ===
plt.figure(figsize=(12, 6))
plt.plot(y_train_inv, label='Actual')
plt.plot(y_pred_train_inv, label='Predicted')
plt.title("Train Set: Predicted vs Actual USD/PKR")
plt.xlabel("Time")
plt.ylabel("USD/PKR Rate")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig("prediction_vs_actual_train.png")
plt.show()

# Future Prediction
def predict_future(model, data, lookback, steps, scaler):
    recent_sequence = data[-lookback:]
    predictions = []
    for _ in range(steps):
        input_seq = np.expand_dims(recent_sequence, axis=0)
        pred_scaled = model.predict(input_seq, verbose=0)[0][0]
        full_pred = np.concatenate([[pred_scaled], recent_sequence[-1, 1:]])
        inv_pred = scaler.inverse_transform([full_pred])[0][0]
        predictions.append(inv_pred)
        next_step = np.append(pred_scaled, recent_sequence[-1, 1:])
        recent_sequence = np.vstack((recent_sequence[1:], next_step))
    return predictions

future_values = predict_future(final_model, scaled_data, lookback, steps=7, scaler=scaler)
print("Future Predictions (USD/PKR):", future_values)